Import modules

In [1]:
import rescal
import scipy
import pandas as pd
import numpy as np

Import node list from node descriptions

In [2]:
nodes = {}

with open("graph/graph_nodes_v2020-12-02.csv") as file:
    file.readline()
    index = 0
    while True:
        line = file.readline()
        node = line.rsplit(",")[0]
        nodes[node] = index
        index += 1
        if not line:
            break
index_range = len(nodes)

Index range is used to handle RESCAL entities formatting (E1..En bottom to top and left to right.)

Import X_"interacts with" and convert to sparse matrix

In [3]:
relationships = {}
k = []
X_tensor = []
ranks = []

with open("graph/graph_edges_v2020-12-02.csv") as file:
    file.readline()
    print("fill tables")
    for line in file.readlines():
        node1, rel, node2 = line.split(",")[0:3]
        if rel not in relationships.keys():
            relationships[rel] = scipy.sparse.dok_matrix((index_range, index_range), dtype=np.float64)
        relationships[rel][index_range - nodes[node1] - 1, nodes[node2]] = np.float64(1)
    print("CSR")
    keys = list(relationships.keys())
    for key in keys:
        k.append(key)
        X_tensor.append(scipy.sparse.csr_matrix(relationships.pop(key)))
    print("ranking")
    for matrix in X_tensor:
        ranks.append(scipy.sparse.csgraph.structural_rank(matrix))
    

fill tables
CSR
ranking


In [4]:
print(k)
print(ranks)

['NA', 'RO:HOM0000017', 'RO:HOM0000020', 'RO:0002434', 'RO:0002206', 'RO:0002200', 'RO:0003304', 'RO:0002327', 'BFO:0000050', 'RO:0002325', 'RO:0003301', 'RO:0002331', 'dc:source', 'GENO:0000222', 'GENO:0000418', 'RO:0002435', 'GENO:0000408', 'RO:0002326', 'RO:0003303', 'GENO:0000840', 'RO:0004013', 'RO:0004016', 'RO:0002607']
[26, 3888, 5961, 2535, 20, 1260, 34, 10, 18, 240, 5, 18, 1, 9, 18, 86, 3, 3, 9, 2, 2, 1, 1]


In [5]:
A, R, f, iteration, execution_times = rescal.als(X_tensor, rank=20, orthogonalize=True)

In [6]:
A

array([[ 1.42030223e-21, -8.53482559e-16,  3.15714866e-16, ...,
         1.40512327e-15,  2.16840464e-15,  3.22658156e-16],
       [-5.18833779e-07, -5.92623186e-07, -8.55969365e-07, ...,
         1.09385205e-07, -1.14534770e-06,  1.02430158e-06],
       [-8.79014361e-07, -9.99683803e-07, -8.47382604e-07, ...,
         8.03177040e-07, -1.83422708e-06,  1.66635198e-06],
       ...,
       [ 3.04782593e-04,  3.60668371e-04, -1.43297626e-03, ...,
         1.42486731e-03, -3.97845839e-04,  3.93440277e-04],
       [ 5.48900797e-05,  6.15585116e-05, -1.16464563e-04, ...,
         1.22588597e-04, -5.84147770e-05,  6.34037935e-05],
       [-7.50605804e-22, -9.92995876e-22,  2.72021168e-21, ...,
        -3.13206362e-21,  9.94958269e-22, -9.36367642e-22]])

In [7]:
rescal.orth(A)

array([[ 0.00000000e+00, -3.89792365e-15,  2.58820743e-15, ...,
         9.50628465e-16,  3.51454976e-15, -2.02615702e-15],
       [-5.18833779e-07, -5.92623186e-07, -8.55969367e-07, ...,
         1.09385204e-07, -1.14534769e-06,  1.02430158e-06],
       [-8.79014361e-07, -9.99683803e-07, -8.47382604e-07, ...,
         8.03177029e-07, -1.83422708e-06,  1.66635197e-06],
       ...,
       [ 3.04782593e-04,  3.60668371e-04, -1.43297626e-03, ...,
         1.42486731e-03, -3.97845839e-04,  3.93440277e-04],
       [ 5.48900797e-05,  6.15585116e-05, -1.16464563e-04, ...,
         1.22588597e-04, -5.84147770e-05,  6.34037935e-05],
       [-7.50605804e-22, -9.92995876e-22,  2.72021168e-21, ...,
        -3.13206362e-21,  9.94958269e-22, -9.36367642e-22]])

In [8]:
R

[array([[ 2.28322614e-04,  3.64582739e-04, -3.76949695e-04,
          9.25472546e-06,  7.62087405e-04,  7.32582620e-03,
         -6.16452506e-03,  8.54776042e-03,  5.60553238e-04,
          4.31953033e-04,  8.64105452e-04, -7.95969027e-04,
         -1.12161007e-02,  5.85538020e-04, -5.68018046e-03,
         -2.62219559e-03,  2.58488908e-04, -1.25256191e-04,
         -2.41767949e-04,  1.97093790e-04],
        [ 2.79401349e-04,  4.46156271e-04, -4.61639222e-04,
          1.14254737e-05,  9.32692344e-04,  8.96800467e-03,
         -7.54584770e-03,  1.04631723e-02,  6.86079082e-04,
          5.28876703e-04,  1.05777213e-03, -9.74286737e-04,
         -1.37296502e-02,  7.16692505e-04, -6.95349491e-03,
         -3.20976168e-03,  3.16443487e-04, -1.53619348e-04,
         -2.96309270e-04,  2.41567039e-04],
        [-3.16582860e-04, -5.05696722e-04,  5.24903461e-04,
         -1.24276539e-05, -1.06154064e-03, -1.01895422e-02,
          8.57799395e-03, -1.18948223e-02, -7.80883456e-04,
         -5.

In [9]:
len(ranks)

23

In [12]:
len(A[0])

20

In [19]:
calc = (A * R[0]) * A.T
pred = []
for rel in calc:
    pred.append(np.sum(rel))
pred

ValueError: operands could not be broadcast together with shapes (20,20) (33818,20) 